In [1]:
import pandas as pd
import numpy as np
import warnings
import regex as re
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
almeria = pd.read_json('../data/datos_aemet_6302A.json')

a_coruna = pd.read_json('../data/datos_aemet_1387.json')

huelva = pd.read_json('../data/datos_aemet_4642E.json')

girona = pd.read_json('../data/datos_aemet_0367.json')

cadiz = pd.read_json('../data/datos_aemet_5973.json')

En las tablas he visto como no todas ellas están estructuradas de la misma forma, intuyo que por ser estaciones diferentes, recogen los datos con diferente estructura. Tendré que ir una a una unificando estructura.

In [3]:
almeria.head()

,fecha,indicativo,p_max,nw_55,w_racha,np_100,nw_91,np_001,w_rec,np_300,p_mes,w_med,np_010,hr,tm_min,ta_max,ts_min,nt_30,ta_min,e,nt_00,ti_max,tm_mes,tm_max,q_max,q_mar,q_med,q_min,n_cub,n_des,n_nub
0,2019-1,6302A,2.4(19),1.0,19/15.6(28),0.0,0.0,4.0,163.0,0.0,4.7,7.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-2,6302A,4.1(01),NaN,NaN,0.0,NaN,2.0,NaN,0.0,4.2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-3,6302A,3.0(30),NaN,NaN,0.0,NaN,3.0,NaN,0.0,6.2,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-4,6302A,17.0(18),0.0,35/14.4(15),1.0,0.0,12.0,154.0,0.0,53.1,8.0,8.0,50.0,7.8,27.7(15),12.6,0.0,4.2(06),77.0,0.0,13.0,13.5,19.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-5,6302A,3.9(03),0.0,08/11.4(23),0.0,0.0,1.0,157.0,0.0,3.9,8.0,1.0,41.0,13.1,32.2(15),18.5,2.0,9.8(03),98.0,0.0,20.2,19.4,25.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
almeria.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       52 non-null     object 
 1   indicativo  52 non-null     object 
 2   p_max       47 non-null     object 
 3   nw_55       43 non-null     float64
 4   w_racha     43 non-null     object 
 5   np_100      47 non-null     float64
 6   nw_91       43 non-null     float64
 7   np_001      47 non-null     float64
 8   w_rec       43 non-null     float64
 9   np_300      47 non-null     float64
 10  p_mes       47 non-null     float64
 11  w_med       45 non-null     float64
 12  np_010      47 non-null     float64
 13  hr          37 non-null     float64
 14  tm_min      37 non-null     float64
 15  ta_max      37 non-null     object 
 16  ts_min      37 non-null     float64
 17  nt_30       37 non-null     float64
 18  ta_min      37 non-null     object 
 19  e           37 non-null     flo

identifico los datos que me van a servir. Hay demasiados nulos en esta tabla, vamos a borrar columnas que no nos sirvan antes de ver los nulos

In [5]:
almeria.columns

Index(['fecha', 'indicativo', 'p_max', 'nw_55', 'w_racha', 'np_100', 'nw_91',
       'np_001', 'w_rec', 'np_300', 'p_mes', 'w_med', 'np_010', 'hr', 'tm_min',
       'ta_max', 'ts_min', 'nt_30', 'ta_min', 'e', 'nt_00', 'ti_max', 'tm_mes',
       'tm_max', 'q_max', 'q_mar', 'q_med', 'q_min', 'n_cub', 'n_des',
       'n_nub'],
      dtype='object')

In [6]:
almeria = almeria[['fecha', 'indicativo', 'p_mes', 'p_max', 'w_med', 
                   'tm_mes', 'tm_max', 'tm_min', 'n_cub']]
#Estas son las columnas que no harán falta para el estudio. Dejaré una leyendo en nuestra base de datos 
#para entender la nomenclatura.

In [7]:
almeria.head()

,fecha,indicativo,p_mes,p_max,w_med,tm_mes,tm_max,tm_min,n_cub
0,2019-1,6302A,4.7,2.4(19),7.0,NaN,NaN,NaN,NaN
1,2019-2,6302A,4.2,4.1(01),NaN,NaN,NaN,NaN,NaN
2,2019-3,6302A,6.2,3.0(30),NaN,NaN,NaN,NaN,NaN
3,2019-4,6302A,53.1,17.0(18),8.0,13.5,19.1,7.8,NaN
4,2019-5,6302A,3.9,3.9(03),8.0,19.4,25.7,13.1,NaN


In [8]:
almeria.isnull().sum()

fecha          0
indicativo     0
p_mes          5
p_max          5
w_med          7
tm_mes        15
tm_max        15
tm_min        15
n_cub         36
dtype: int64

In [9]:
almeria.shape

(52, 9)

Me doy cuenta que no concuerdan los tres años de datos con las filas que deben haber en mi DF. Es porque hay una fila resumen por cada año, ej: 2019-13. Las borro.

In [10]:
#Creo una función porque tendré que borrar estas mismas filas en el resto de tablas de provincias
def borrar_filas(df):
    lista_filas = [12, 25, 38, 51]
    df_nuevo = df.drop(lista_filas)
    df_nuevo = df_nuevo.reset_index(drop=True)
    return df_nuevo

In [11]:
almeria = almeria.apply(borrar_filas)

In [12]:
almeria.shape

(48, 9)

In [13]:
almeria.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       48 non-null     object 
 1   indicativo  48 non-null     object 
 2   p_mes       44 non-null     float64
 3   p_max       44 non-null     object 
 4   w_med       43 non-null     float64
 5   tm_mes      37 non-null     float64
 6   tm_max      37 non-null     float64
 7   tm_min      37 non-null     float64
 8   n_cub       15 non-null     float64
dtypes: float64(6), object(3)
memory usage: 3.5+ KB


terminamos. dejamos los nulos puesto que si los cambiamos a cualquier valor no nos refleja la info que nos da la tabla. Hacemos lo mismo con el resto.

In [20]:
def borrar_filas_columnas(df):
    
    columnas_deseadas = ['fecha', 'indicativo', 'p_mes', 'p_max', 'w_med', 'tm_mes', 'tm_max', 'tm_min', 'n_cub']
    
    if isinstance(df, pd.Series):
        df = df.to_frame().T
    
    for columna in columnas_deseadas:#Esto evita error si no existe una de esta cols en el DF.
        if columna not in df.columns:
            df[columna] = None #Crea una columna de nulos
    
    lista_filas = [12, 25, 38, 51]
    df_nuevo = df.drop(lista_filas)
    df_nuevo = df_nuevo.reset_index(drop=True)
    return df_nuevo

In [22]:
a_coruna.columns

Index(['fecha', 'indicativo', 'p_max', 'glo', 'hr', 'n_gra', 'n_fog', 'inso',
       'q_max', 'nw_55', 'q_mar', 'q_med', 'tm_min', 'ta_max', 'ts_min',
       'nt_30', 'nv_0050', 'w_racha', 'np_100', 'p_sol', 'nw_91', 'ts_20',
       'np_001', 'ta_min', 'w_rec', 'e', 'np_300', 'nv_1000', 'evap', 'p_mes',
       'n_llu', 'n_tor', 'ts_10', 'w_med', 'nt_00', 'ti_max', 'n_nie',
       'tm_mes', 'tm_max', 'nv_0100', 'q_min', 'np_010', 'n_cub', 'n_des',
       'n_nub'],
      dtype='object')

In [23]:
a_coruna = a_coruna[['fecha', 'indicativo', 'p_mes', 'p_max', 'w_med', 
                   'tm_mes', 'tm_max', 'tm_min', 'n_cub']]

In [25]:
a_coruna.shape

(52, 9)

In [26]:
a_coruna = a_coruna.apply(borrar_filas)

In [27]:
a_coruna.shape

(48, 9)

In [28]:
cadiz.columns

Index(['fecha', 'indicativo', 'p_max', 'glo', 'hr', 'q_max', 'nw_55', 'q_mar',
       'q_med', 'tm_min', 'ta_max', 'ts_min', 'nt_30', 'w_racha', 'np_100',
       'nw_91', 'np_001', 'ta_min', 'w_rec', 'e', 'np_300', 'p_mes', 'w_med',
       'nt_00', 'ti_max', 'tm_mes', 'tm_max', 'q_min', 'np_010', 'evap',
       'n_gra', 'n_fog', 'n_llu', 'n_tor', 'n_nie', 'n_cub', 'n_des', 'n_nub'],
      dtype='object')

In [29]:
cadiz = cadiz[['fecha', 'indicativo', 'p_mes', 'p_max', 'w_med', 
                   'tm_mes', 'tm_max', 'tm_min', 'n_cub']]

In [30]:
cadiz.shape

(52, 9)

In [31]:
cadiz = cadiz.apply(borrar_filas)

In [32]:
cadiz.shape

(48, 9)

In [33]:
huelva.columns

Index(['fecha', 'indicativo', 'p_max', 'glo', 'hr', 'n_gra', 'n_fog', 'inso',
       'q_max', 'nw_55', 'q_mar', 'q_med', 'tm_min', 'ta_max', 'ts_min',
       'nt_30', 'w_racha', 'np_100', 'p_sol', 'nw_91', 'ts_20', 'np_001',
       'ta_min', 'w_rec', 'e', 'np_300', 'p_mes', 'n_llu', 'n_tor', 'w_med',
       'nt_00', 'ti_max', 'n_nie', 'tm_mes', 'tm_max', 'q_min', 'np_010',
       'evap', 'n_cub', 'nv_0050', 'n_des', 'n_nub', 'nv_1000', 'nv_0100'],
      dtype='object')

In [34]:
huelva = huelva[['fecha', 'indicativo', 'p_mes', 'p_max', 'w_med', 
                   'tm_mes', 'tm_max', 'tm_min', 'n_cub']]

In [35]:
huelva.shape

(52, 9)

In [36]:
huelva = huelva.apply(borrar_filas)

In [37]:
huelva.shape

(48, 9)

In [38]:
girona.columns

Index(['fecha', 'indicativo', 'p_max', 'n_cub', 'glo', 'hr', 'n_gra', 'n_fog',
       'inso', 'q_max', 'nw_55', 'q_mar', 'q_med', 'tm_min', 'ta_max',
       'ts_min', 'nt_30', 'nv_0050', 'n_des', 'w_racha', 'np_100', 'n_nub',
       'p_sol', 'nw_91', 'np_001', 'ta_min', 'w_rec', 'e', 'np_300', 'nv_1000',
       'p_mes', 'n_llu', 'n_tor', 'w_med', 'nt_00', 'ti_max', 'n_nie',
       'tm_mes', 'tm_max', 'nv_0100', 'q_min', 'np_010'],
      dtype='object')

In [39]:
girona = girona[['fecha', 'indicativo', 'p_mes', 'p_max', 'w_med', 
                   'tm_mes', 'tm_max', 'tm_min', 'n_cub']]

In [40]:
girona = girona.apply(borrar_filas)

In [41]:
girona.shape

(48, 9)

Modifico los tipos de cada columna para que todas tengan la misma estructura antes de concatenarlas.

In [48]:
a_coruna.indicativo = a_coruna.indicativo.astype(object)

In [50]:
girona.indicativo = girona.indicativo.astype(object)
girona.w_med = girona.w_med.astype(float)

In [52]:
cadiz.indicativo = cadiz.indicativo.astype(object)

In [44]:
huelva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       48 non-null     object 
 1   indicativo  48 non-null     object 
 2   p_mes       48 non-null     float64
 3   p_max       48 non-null     object 
 4   w_med       47 non-null     float64
 5   tm_mes      48 non-null     float64
 6   tm_max      48 non-null     float64
 7   tm_min      48 non-null     float64
 8   n_cub       15 non-null     float64
dtypes: float64(6), object(3)
memory usage: 3.5+ KB


In [53]:
cadiz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       48 non-null     object 
 1   indicativo  48 non-null     object 
 2   p_mes       48 non-null     float64
 3   p_max       48 non-null     object 
 4   w_med       47 non-null     float64
 5   tm_mes      48 non-null     float64
 6   tm_max      48 non-null     float64
 7   tm_min      48 non-null     float64
 8   n_cub       14 non-null     float64
dtypes: float64(6), object(3)
memory usage: 3.5+ KB


In [51]:
girona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       48 non-null     object 
 1   indicativo  48 non-null     object 
 2   p_mes       48 non-null     float64
 3   p_max       48 non-null     object 
 4   w_med       48 non-null     float64
 5   tm_mes      47 non-null     float64
 6   tm_max      47 non-null     float64
 7   tm_min      47 non-null     float64
 8   n_cub       45 non-null     float64
dtypes: float64(6), object(3)
memory usage: 3.5+ KB


In [49]:
a_coruna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       48 non-null     object 
 1   indicativo  48 non-null     object 
 2   p_mes       48 non-null     float64
 3   p_max       48 non-null     object 
 4   w_med       43 non-null     float64
 5   tm_mes      48 non-null     float64
 6   tm_max      48 non-null     float64
 7   tm_min      48 non-null     float64
 8   n_cub       5 non-null      float64
dtypes: float64(6), object(3)
memory usage: 3.5+ KB


In [47]:
almeria.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       48 non-null     object 
 1   indicativo  48 non-null     object 
 2   p_mes       44 non-null     float64
 3   p_max       44 non-null     object 
 4   w_med       43 non-null     float64
 5   tm_mes      37 non-null     float64
 6   tm_max      37 non-null     float64
 7   tm_min      37 non-null     float64
 8   n_cub       15 non-null     float64
dtypes: float64(6), object(3)
memory usage: 3.5+ KB


In [54]:
tabla_clima = pd.concat([almeria,a_coruna,huelva,girona,cadiz])

In [55]:
tabla_clima.shape

(240, 9)

In [58]:
tabla_clima.head(50)

,fecha,indicativo,p_mes,p_max,w_med,tm_mes,tm_max,tm_min,n_cub
0,2019-1,6302A,4.7,2.4(19),7.0,NaN,NaN,NaN,NaN
1,2019-2,6302A,4.2,4.1(01),NaN,NaN,NaN,NaN,NaN
2,2019-3,6302A,6.2,3.0(30),NaN,NaN,NaN,NaN,NaN
3,2019-4,6302A,53.1,17.0(18),8.0,13.5,19.1,7.8,NaN
4,2019-5,6302A,3.9,3.9(03),8.0,19.4,25.7,13.1,NaN
5,2019-6,6302A,2.2,2.2(13),11.0,NaN,NaN,NaN,NaN
6,2019-7,6302A,0.3,0.3(07),11.0,26.4,32.8,19.9,NaN
7,2019-8,6302A,1.9,1.8(31),10.0,26.2,32.6,19.7,NaN
8,2019-9,6302A,44.6,28.5(12),7.0,22.1,27.7,16.6,NaN
9,2019-10,6302A,10.9,4.8(19),5.0,18.8,24.8,12.7,NaN


In [59]:
tabla_clima.to_csv('../data/tablas_limpias/tabla_clima.csv', index=False)